In [2]:
import numpy as np
import awkward as ak
np.random.seed(1337)  # for reproducibility

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

from sklearn.metrics import roc_curve, auc

import matplotlib.pyplot as plt

import utils as pic

%load_ext autoreload
%autoreload 2

In [3]:
lr_init     = 1.e-3    # Initial learning rate  
batch_size  = 64       # Training batch size
train_size  = 1024     # Training size
valid_size  = 1024     # Validation size
test_size   = 1024     # Test size
epochs      = 20       # Number of epochs
doGPU       = False    # Use GPU

In [4]:
# Set range of training set
train_start, train_stop = 0, train_size
assert train_stop > train_start
assert (len(pic.decays)*train_size) % batch_size == 0
X_train, y_train = pic.load_data(train_start,train_stop)

# Set range of validation set
valid_start, valid_stop = 160000, 160000+valid_size
assert valid_stop  >  valid_start
assert valid_start >= train_stop
X_valid, y_valid = pic.load_data(valid_start,valid_stop)

# Set range of test set
test_start, test_stop = 204800, 204800+test_size
assert test_stop  >  test_start
assert test_start >= valid_stop
X_test, y_test = pic.load_data(test_start,test_stop)

samples_requested = len(pic.decays) * (train_size + valid_size + test_size)
samples_available = len(y_train) + len(y_valid) + len(y_test)
assert samples_requested == samples_available

In [5]:
X_train = pic.remove_empty_pixels(X_train)
X_train.shape

(2048, 32, 32, 2)

In [6]:
X_e_train,X_t_train,maxframes = pic.timeordered(X_train,cumulative=False)
X_e_train = np.where(np.isnan(X_e_train),0,X_e_train)
X_e_train = np.where(np.isnan(X_t_train)[:,:,None],-69420,X_e_train.reshape(2*train_size,-1,32*32)).reshape(X_e_train.shape)
y_train = to_categorical(y_train)

In [7]:
X_e_train.shape

(2048, 78, 32, 32, 1)

In [8]:
X_e_test,X_t_test,maxframes = pic.timeordered(X_test,cumulative=False)
X_e_test = np.where(np.isnan(X_e_test),0,X_e_test)
X_e_test = np.where(np.isnan(X_t_test)[:,:,None],-69420,X_e_test.reshape(2*test_size,-1,32*32)).reshape(X_e_test.shape)
y_test = to_categorical(y_test)

In [9]:
model = keras.Sequential()

model.add(layers.Masking(mask_value=-69420, input_shape=(None, 32,32, 1)))
model.add(layers.ConvLSTM2D(64,3,activation='relu',padding='same'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(32,3,activation='relu',padding='same'))
model.add(layers.MaxPool2D())
model.add(layers.Flatten())
model.add(layers.Dense(10,activation='relu'))
model.add(layers.Dense(2,activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 32, 32, 1)   0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 32, 32, 64)        150016    
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 16, 16, 32)        18464     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 32)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0

In [10]:
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

In [11]:
y_pred_prefit = model.predict(X_e_test)

In [10]:
from sklearn.metrics import accuracy_score

accuracy_score(np.argmax(y_test[:100],axis=-1),np.argmax(y_pred_prefit,axis=-1))

0.81

In [11]:
history = model.fit(
    X_e_train, y_train,
    validation_split=0.2,
    epochs=3,
    batch_size=50,
    shuffle=True,
    verbose=1
)

Epoch 1/3


In [1]:
model

NameError: name 'model' is not defined